# Prepare HydroFabric Map Layers

**Authors**: 
- Irene Garousi-Nejad <igarousi@cuahsi.org>, 
- Tony Castronova <acastronova@cuahsi.org>

**Last Updated**: 

**Description**:  

**Software Requirements**:

> Conda: 22.9.0  \
> Python: 3.9.16  \
> wget: 3.2  \
> pandas: 2.0.0  \
> geopandas: 0.12.2  \
> fiona:  1.9.1

---

In [1]:
import boto3
import fsspec
import pandas
import geopandas

ModuleNotFoundError: No module named 'boto3'

## Merge Boundaries of all VPUs into Regions

In [ ]:
%%time

geoms = []
s3 = boto3.resource('s3')
s3 = fsspec.filesystem('s3', anon=True)
for vpu in ['01', '02','03N','03S','03W', '04','05','06','07','08','09','10L','10U','11','12','13','14','15','16','17','18']:
    print(f'Processing VPU: {vpu}', end='...', flush=True)
    with s3.open(f's3://nextgen-hydrofabric/pre/nextgen_{vpu}.gpkg') as f:    
        # Read the file-like object into a GeoDataFrame
        gdf_divide = geopandas.read_file(f, layer='divides')
        gdf_divide['geometry'] = gdf_divide.buffer(0.01) # this is necessary to ensure that geometries are valid
        geom = gdf_divide.dissolve()
        
        # save the result in a list
        geom['VPU'] = vpu
        geom.drop(columns=['id','areasqkm','type','toid'], inplace=True)
        geoms.append(geom)
    print('done')

Processing VPU: 16...done
CPU times: user 34.8 s, sys: 3.47 s, total: 38.3 s
Wall time: 1min 30s


In [55]:
gdf = geopandas.GeoDataFrame(pandas.concat(geoms, ignore_index=True))

In [58]:
gdf.to_file('vpu_boundaries.shp', driver='ESRI Shapefile')

## Save Boundaries for Each VPU

In [3]:
s3 = boto3.resource('s3')
s3 = fsspec.filesystem('s3', anon=True)


#for vpu in ['01', '02','03N','03S','03W', '04','05','06','07','08','09','10L','10U','11','12','13','14','15','16','17','18']:
for vpu in ['16']:
    print(f'Processing VPU: {vpu}', end='...', flush=True)
    with s3.open(f's3://nextgen-hydrofabric/v1.2/nextgen_{vpu}.gpkg') as f:    
        # Read the file-like object into a GeoDataFrame
        gdf = geopandas.read_file(f, layer='divides')
        gdf.to_file(f'{vpu}_boundaries.shp', driver='ESRI Shapefile')
    print('done')
    



Processing VPU: 16...done
